# Daftar Isi
## A. Pengumpulan Data
### 1. Inisiasi link
### 2. web crawling
## B. Preprocessing Data
### 1. Preprocessing Text
### 2. Pembobotan TF IDF
## C. Pengujian Algoritma
### 1. Penentuan Nilai K terbaik
### 2. Penentuan Perbandingan data Training dan Testing terbaik
### 3. K-Fold Cross Validation

#################################

# Klasifikasi Berita

## A. Pengumpulan Data

### 1. Inisiasi link

In [17]:
daftarLinkKetegori = [
    "https://indeks.kompas.com/?site=bola&page=",
    "https://indeks.kompas.com/?site=lifestyle&page=",
    "https://indeks.kompas.com/?site=sains&page"
]

### 2. web crawling

In [23]:
import urllib
import urllib.request
from bs4 import BeautifulSoup

import pandas as pd

# fugnsi request
def make_soup(url):
    thepage = urllib.request.urlopen(url)
    soupdata = BeautifulSoup(thepage, "html.parser")
    return soupdata

#Jumlah Kategori
jmlKategori = len(daftarLinkKetegori)

#jumlah Item Per Kategori
jmlItem = 0
hitung = 0

dataCrawling = pd.DataFrame([], columns=['link', 'content', 'label'])

# crawling link sama beritanya
# loop sebanyak indek/kelas
for kategori in range(3):
    
    # loop sebanyak 10 page (1 page = 16 item berita)
    for i in range(1, 10):
        
        #request link
        soup = make_soup(daftarLinkKategori[kategori] + str(i))
        
        #loop sebanyak  16 item berita
        for record in soup.find_all("div", "article__list clearfix"):
            print(hitung, end=" ")
            
            #crawling link
            link = record.a.get("href") #link
            
            #crawling konten
            soup = make_soup(link) # request iis berita data
            content = soup.find("div", "read__content").text # content
            
            label = kategori # label
            
            row = pd.DataFrame([[link, content, label]], columns=dataCrawling.columns)
            dataCrawling = pd.concat([dataCrawling, row], ignore_index=True)
            
            hitung += 1
            
dataCrawling

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

,link,content,label
0,https://www.kompas.com/sports/read/2021/08/14/...,\n\nKOMPAS.com - Sama seperti cabang olahraga ...,0
1,https://www.kompas.com/sports/read/2021/08/14/...,"\n\nTOKYO, KOMPAS.com - Pada Paralimpik Tokyo ...",0
2,https://www.kompas.com/motogp/read/2021/08/14/...,\n\nKOMPAS.com - Jadwal MotoGP Austria 2021 ha...,0
3,https://www.kompas.com/sports/read/2021/08/14/...,"\n\nJAKARTA, KOMPAS.com - Pelaksanaan protokol...",0
4,https://bola.kompas.com/read/2021/08/14/090000...,"\nKOMPAS.com - Kasta tertinggi Liga Jerman, Bu...",0
...,...,...,...
400,https://www.kompas.com/sains/read/2021/08/13/1...,\n\nKOMPAS.com - Sepanjang eksplorasi laut dal...,2
401,https://www.kompas.com/sains/read/2021/08/13/1...,"\n\nKOMPAS.com - Seiring perkembangan zaman, m...",2
402,https://www.kompas.com/sains/read/2021/08/13/1...,\n\nKOMPAS.com - Sebuah studi yang dipublikasi...,2
403,https://www.kompas.com/sains/read/2021/08/13/1...,\n\nKOMPAS.com - Sebagian orang mengalami keri...,2


In [24]:
## save data
dataCrawling.to_csv('dataBerita.csv', index=False)

## B. Preprocessing Data

### 1. Preprocessing Text

In [6]:
#Import Library

#nltk.download('stopwords') #jika belum ada
import nltk

#library casefolding
import re
import string

#library stemming
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# create stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

#Library Filtering/stopword
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, ArrayDictionary, StopWordRemover
from nltk.corpus import stopwords

In [7]:
#load data crawling
itemBerita = pd.read_csv("dataBerita.csv") 
itemBerita

,link,content,label
0,https://www.kompas.com/sports/read/2021/08/14/...,\n\nKOMPAS.com - Sama seperti cabang olahraga ...,0
1,https://www.kompas.com/sports/read/2021/08/14/...,"\n\nTOKYO, KOMPAS.com - Pada Paralimpik Tokyo ...",0
2,https://www.kompas.com/motogp/read/2021/08/14/...,\n\nKOMPAS.com - Jadwal MotoGP Austria 2021 ha...,0
3,https://www.kompas.com/sports/read/2021/08/14/...,"\n\nJAKARTA, KOMPAS.com - Pelaksanaan protokol...",0
4,https://bola.kompas.com/read/2021/08/14/090000...,"\nKOMPAS.com - Kasta tertinggi Liga Jerman, Bu...",0
...,...,...,...
400,https://www.kompas.com/sains/read/2021/08/13/1...,\n\nKOMPAS.com - Sepanjang eksplorasi laut dal...,2
401,https://www.kompas.com/sains/read/2021/08/13/1...,"\n\nKOMPAS.com - Seiring perkembangan zaman, m...",2
402,https://www.kompas.com/sains/read/2021/08/13/1...,\n\nKOMPAS.com - Sebuah studi yang dipublikasi...,2
403,https://www.kompas.com/sains/read/2021/08/13/1...,\n\nKOMPAS.com - Sebagian orang mengalami keri...,2


In [60]:
#Algoritma Preprocessing Text

from IPython import display

#cuma count up
hitung = 0

def preprocessing(isi):
    #a. CaseFolding (lowercase, angka, tanda, whitespace)
    
    #Bersihin \n
    isi = isi.replace('\n', " ").replace('xa0',' ')
    
    #lower case
    isi = re.sub(r"\d+", " ", isi)
    
    #hapus tanda baca
    isi = isi.translate(str.maketrans("", "", string.punctuation))
    
    #hapus whitespace (karakter kosong)
    isi = isi.strip()
    
    #b. stemming
    isi = stemmer.stem(isi)
    
    #c. stopwords/filtering
    wordNlt = list(set(stopwords.words('indonesian'))) #load stopword
    wordSastrawi = StopWordRemoverFactory().get_stop_words() #load default stopwrod
    wordNew = ['kompascom', 'baca', 'isi'] #menambah stopword
    
    wordAll = wordNew + wordNlt + wordSastrawi # menggabungkan list stopword
    dictionary = ArrayDictionary(wordAll)
    
    isi = StopWordRemover(dictionary).remove(isi) # filter pertama
    isi = StopWordRemover(dictionary).remove(isi) # filter kedua
    
    #3. Tokenizing
    isi = isi.split()
    
    display.clear_output(wait=True)
    
    global hitung
    hitung += 1
    print(hitung, " item preprocessed", end='')
    
    return isi

#daftar isi semua berita sudah preprocessing
itemBerita['content-pp'] = itemBerita['content'].apply(preprocessing)
itemBerita

405  item preprocessed

,link,content,label,content-pp
0,https://www.kompas.com/sports/read/2021/08/14/...,\n\nKOMPAS.com - Sama seperti cabang olahraga ...,0,"[cabang, olahraga, softball, milik, taktik, ta..."
1,https://www.kompas.com/sports/read/2021/08/14/...,"\n\nTOKYO, KOMPAS.com - Pada Paralimpik Tokyo ...",0,"[tokyo, paralimpik, tokyo, kali, atlet, disabi..."
2,https://www.kompas.com/motogp/read/2021/08/14/...,\n\nKOMPAS.com - Jadwal MotoGP Austria 2021 ha...,0,"[jadwal, motogp, austria, sabtu, saji, sesi, l..."
3,https://www.kompas.com/sports/read/2021/08/14/...,"\n\nJAKARTA, KOMPAS.com - Pelaksanaan protokol...",0,"[jakarta, laksana, protokol, sehat, covid, sel..."
4,https://bola.kompas.com/read/2021/08/14/090000...,"\nKOMPAS.com - Kasta tertinggi Liga Jerman, Bu...",0,"[kasta, liga, jerman, bundesliga, rupa, salah,..."
...,...,...,...,...
400,https://www.kompas.com/sains/read/2021/08/13/1...,\n\nKOMPAS.com - Sepanjang eksplorasi laut dal...,2,"[eksplorasi, laut, ilmuwan, lihat, ubah, bentu..."
401,https://www.kompas.com/sains/read/2021/08/13/1...,"\n\nKOMPAS.com - Seiring perkembangan zaman, m...",2,"[iring, kembang, zaman, muncul, alat, masak, b..."
402,https://www.kompas.com/sains/read/2021/08/13/1...,\n\nKOMPAS.com - Sebuah studi yang dipublikasi...,2,"[buah, studi, publikasi, jurnal, iscience, ker..."
403,https://www.kompas.com/sains/read/2021/08/13/1...,\n\nKOMPAS.com - Sebagian orang mengalami keri...,2,"[orang, alami, keringat, nyata, keringat, mala..."


### 2. Pembobotan TF-IDF


In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

V = TfidfVectorizer()
# for title in steming_title:
a =  V.fit_transform([' '.join(title) for title in itemBerita['content-pp'] ])
vectorized_title = a

feature_names = V.get_feature_names()
dense = a.todense()
denselist = dense.tolist()

dfTFIDF = pd.DataFrame(denselist, columns=feature_names)
dfTFIDF

,aarhus,aaron,abad,abadi,abai,abalabal,abbey,abc,abdi,abdul,...,zodiak,zola,zona,zonat,zone,zoologi,zoonosis,zoto,zouma,zphrthl
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400,0.0,0.0,0.044209,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.045124,0.0,0.0,0.0,0.0
401,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
402,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
403,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0


## C. Pengujian Algoritma

In [62]:
# Import Library KNN

import pprint
import operator

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.datasets import load_iris, load_breast_cancer
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator, ClassifierMixin

In [63]:
# Function Algoritma FKNN

class FuzzyKNN(BaseEstimator, ClassifierMixin):
    def __init__(self, k=3, plot=False):
        self.k = k
        self.plot = plot
        
        
    def fit(self, X, y=None):
        self._check_params(X,y)
        self.X = X
        self.y = y
        
        self.xdim = len(self.X[0])
        self.n = len(y)
        
        classes = list(set(y))
        classes.sort()
        self.classes = classes
        
        self.df = pd.DataFrame(self.X)
        self.df['y'] = self.y
        
        self.memberships = self._compute_memberships()
        
        self.df['membership'] = self.memberships
        
        self.fitted_ = True
        return self
    
    
    def predict(self, X):
        if self.fitted_ == None:
            raise Exception('predict() called before fit()')
        else:
            m = 2
            y_pred = []
            
            for x in X:
                neighbors = self._find_k_nearest_neighbors(pd.DataFrame.copy(self.df), x)
                
                votes = {}
                for c in self.classes:
                    den = 0
                    for n in range(self.k):
                        dist = np.linalg.norm(x - neighbors.iloc[n,0:self.xdim])
                        den += 1 / (dist ** (2 / (m-1)))
                    
                    neighbors_votes = []
                    for n in range(self.k):
                        dist = np.linalg.norm(x - neighbors.iloc[n,0:self.xdim])
                        num = (neighbors.iloc[n].membership[c]) / (dist ** (2 / (m-1)))
                        
                        vote = num/den
                        neighbors_votes.append(vote)
                    votes[c] = np.sum(neighbors_votes)
                    
                pred = max(votes.items(), key=operator.itemgetter(1))[0]
                y_pred.append((pred, votes))
                
            return y_pred
        
        
    def score(self, X, y):
        if self.fitted_ == None:
            raise Exception('score() called before fit()')
        else:
            predictions = self.predict(X)
            y_pred = [t[0] for t in predictions]
            confidences = [t[1] for t in predictions]
            
            return accuracy_score(y_pred=y_pred, y_true=y)
    
        
    def _find_k_nearest_neighbors(self, df, x):
        X = df.iloc[:,0:self.xdim].values
        
        df['distances'] = [np.linalg.norm(X[i] - x) for i in range(self.n)]
        
        df.sort_values(by='distances', ascending=True, inplace=True)
        neighbors = df.iloc[0:self.k]
        
        return neighbors

                
    def _get_counts(self, neighbors):
        groups = neighbors.groupby('y')
        counts = {group[1]['y'].iloc[0]:group[1].count()[0] for group in groups}
        
        return counts
        
        
    def _compute_memberships(self):
        memberships = []
        for i in range(self.n):
            x = self.X[i]
            y = self.y[i]
            
            neighbors = self._find_k_nearest_neighbors(pd.DataFrame.copy(self.df), x)
            counts = self._get_counts(neighbors)
        
            membership = dict()
            for c in self.classes:
                try:
                    uci = 0.49 * (counts[c] / self.k)
                    if c == y:
                        uci += 0.51
                    membership[c] = uci
                except:
                    membership[c] = 0
                    
            memberships.append(membership)
        return memberships
        
        
    def _check_params(self, X, y):
        if type(self.k) != int:
            raise Exception('"k" should have type int')
        elif self.k >= len(y):
            raise Exception('"k" should be less than no of feature sets')
#         elif self.k % 2 == 0:
#             raise Exception('"k" should be odd')
            
        if type(self.plot) != bool:
            raise Exception('"plot" should have type bool')

In [64]:
# Inisiasi data
X = dfTFIDF.values.tolist()
y = itemBerita['label'].tolist()

### 1. Penentuan Nilai k Terbaik

In [67]:
# import library
import numpy as np
from sklearn.model_selection import train_test_split

# pembagian data
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1, random_state=1)

scores = []
list_k = []

for i in range(1, 21):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(xTrain, yTrain)
    list_k.append(i)
    scores.append(knn.score(xTest, yTest))

In [68]:
scores

[0.926829268292683,
 0.9512195121951219,
 1.0,
 0.9512195121951219,
 0.926829268292683,
 0.926829268292683,
 0.9512195121951219,
 0.926829268292683,
 0.9024390243902439,
 0.9024390243902439,
 0.8536585365853658,
 0.8536585365853658,
 0.8536585365853658,
 0.7804878048780488,
 0.7804878048780488,
 0.8048780487804879,
 0.8048780487804879,
 0.8048780487804879,
 0.8048780487804879,
 0.8048780487804879]

### 2. Penentuan Perbandingan Data Training dan Testing tertinggi

In [69]:
list_banding = []
scores_banding = []

k = 3

for i in range(5, 0, -1):
    xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=i/10, random_state=1)
    
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(xTrain, yTrain)
    list_banding.append(i/10)
    scores_banding.append(knn.score(xTest, yTest))


In [70]:
scores_banding

[0.9211822660098522,
 0.9506172839506173,
 0.9672131147540983,
 0.9629629629629629,
 1.0]

### 3. Pengujian K-Fold Cross Validation

In [71]:
X = dfTFIDF.values.tolist()
y = itemBerita['label'].tolist()

k = 3
knn = KNeighborsClassifier(n_neighbors=k)
fknn = FuzzyKNN(k=k)

print(cross_val_score(cv=10, estimator=knn, X=X, y=y))
print(cross_val_score(cv=10, estimator=fknn, X=X, y=y))

[0.97560976 1.         1.         0.92682927 0.92682927 0.925
 0.85       0.975      0.975      0.925     ]


<ipython-input-63-6df2b38d04e7>:47: RuntimeWarning: divide by zero encountered in double_scalars
  den += 1 / (dist ** (2 / (m-1)))
<ipython-input-63-6df2b38d04e7>:52: RuntimeWarning: invalid value encountered in double_scalars
  num = (neighbors.iloc[n].membership[c]) / (dist ** (2 / (m-1)))
<ipython-input-63-6df2b38d04e7>:52: RuntimeWarning: divide by zero encountered in double_scalars
  num = (neighbors.iloc[n].membership[c]) / (dist ** (2 / (m-1)))
<ipython-input-63-6df2b38d04e7>:54: RuntimeWarning: invalid value encountered in double_scalars
  vote = num/den


[0.63414634 0.65853659 0.65853659 0.58536585 0.58536585 0.6
 0.525      0.65       0.65       0.6       ]


In [74]:
X = dfTFIDF.values.tolist()
y = itemBerita['label'].tolist()

k = 3
knn = KNeighborsClassifier(n_neighbors=k)
fknn = FuzzyKNN(k=k)

# print(cross_val_score(cv=10, estimator=knn, X=X, y=y))
print(cross_val_score(cv=10, estimator=fknn, X=X, y=y))

<ipython-input-63-6df2b38d04e7>:47: RuntimeWarning: divide by zero encountered in double_scalars
  den += 1 / (dist ** (2 / (m-1)))
<ipython-input-63-6df2b38d04e7>:52: RuntimeWarning: invalid value encountered in double_scalars
  num = (neighbors.iloc[n].membership[c]) / (dist ** (2 / (m-1)))
<ipython-input-63-6df2b38d04e7>:52: RuntimeWarning: divide by zero encountered in double_scalars
  num = (neighbors.iloc[n].membership[c]) / (dist ** (2 / (m-1)))
<ipython-input-63-6df2b38d04e7>:54: RuntimeWarning: invalid value encountered in double_scalars
  vote = num/den


[0.63414634 0.65853659 0.65853659 0.58536585 0.58536585 0.6
 0.525      0.65       0.65       0.6       ]
